In [1]:
import numpy as np
import datetime

In [100]:
file_path = r'C:\Users\81004\Desktop\laso4-ovito.xyz'

In [102]:
# 输出实时时间
def now_time():
    # 获取当前时间
    current_time = datetime.datetime.now()
    # 输出当前时间
    print("当前时间是:", current_time)

# 晶体

In [105]:
# 读取xyz文件并按照给定条件筛选坐标
def read_xyz_file_with_conditions(filename):
    coordinates = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for line in lines[2:]:  # 从第三行开始读取坐标数据
            parts = line.split()
            if len(parts) >= 4:
                atom = parts[0]
                #  转换一下单位 
                x = float(parts[1])
                y = float(parts[2])
                z = float(parts[3])
                coordinates.append((atom, x, y, z))
    return coordinates

# 使用示例
filename = file_path  # 替换为你的xyz文件名
cry_xyz = read_xyz_file_with_conditions(filename)

In [107]:
# 原始坐标列表
coordinates = cry_xyz
# 晶体中原子的计数
num_cry_dic = create_dic = {'O':0,'Si':0,'Al':0,'Li':0}
# 平移的距离
a, b, c = 0, 0, 0.0  # 替换为你需要的平移值
# 初始化最大最小值
max_x = max_y = max_z = float('-inf')
min_x = min_y = min_z = float('inf')

new_cry_xyz = []

# 平移坐标并寻找最大最小值
for atom, x, y, z in coordinates:
    # 对每个坐标平移
    new_x = x + a
    new_y = y + b
    new_z = z + c
    new_cry_xyz.append([atom,new_x,new_y,new_z])
    # 对晶体在的原子计数
    num_cry_dic[atom] += 1
    # 更新最大值
    if new_x > max_x:
        max_x = new_x
    if new_y > max_y:
        max_y = new_y
    if new_z > max_z:
        max_z = new_z
    
    # 更新最小值
    if new_x < min_x:
        min_x = new_x
    if new_y < min_y:
        min_y = new_y
    if new_z < min_z:
        min_z = new_z

# 输出平移后所有原子中最大的和最小的 x, y, z 值
print("晶体的初始xyz范围：")
print(f"x 值: {min_x} -- {max_x}")
print(f"y 值: {min_y} -- {max_y}")
print(f"z 值: {min_z} -- {max_z}")
print("晶体的原子数：")
print(len(cry_xyz))
print(num_cry_dic)

晶体的初始xyz范围：
x 值: -20.0734325625 -- 19.8450745012
y 值: -9.2715213548 -- 17.6827887711
z 值: -11.099709465 -- 21.2621265677
晶体的原子数：
1890
{'O': 1080, 'Si': 270, 'Al': 270, 'Li': 270}


charge_dic = {'O':0,'Li':0,'Nb':0}

def get_charge(atom_list):
    sum_charge = 0
    for i_atom in atom_list:
        temp_type = i_atom[0]
        sum_charge += charge_dic[temp_type]
    print('总电荷数： {}'.format(round(sum_charge,1)))
# 
get_charge(new_cry_xyz)

In [110]:
num_cry_dic

{'O': 1080, 'Si': 270, 'Al': 270, 'Li': 270}

# 变换坐标

In [115]:
# 原有等效大小的立方 a b c
a = 5.35292
b = 4.63576     # np.sin(np.deg2rad(120.0000000000))*10
c = 11.0997
#########
times_a = 5
times_b = 6
times_c = 3
##############
len_x,len_y,len_z = a*times_a,b*times_b,c*times_c
box_size = np.array([len_x,len_y,len_z])
print(box_size)
print(b)

[26.7646  27.81456 33.2991 ]
4.63576


In [117]:
new_cry_xyz[4]

['Li', -5.35291535, -9.2715213548, -9.2497578871]

In [119]:
(len_x+len_y+len_z)/3/2

14.646376666666669

In [121]:
# 然后变换坐标  让结合面放在四周 而非中间
tran_half_d = (len_x+len_y+len_z)/(3*2)
print(f'将会把接合面转移一半：{tran_half_d}')
#######
cube_list = []
for i_type,x,y,z in new_cry_xyz:
    temp_list = np.array([x,y,z])
    temp_list = (temp_list+tran_half_d)%box_size
    cube_list.append([i_type,temp_list[0],temp_list[1],temp_list[2]])

将会把接合面转移一半：14.646376666666669


In [123]:
cube_list[4]

['Li', 9.293461316666669, 5.374855311866668, 5.396618779566669]

# 当遇到xyz小于0的，先全部给它加上 

In [127]:
def move_to_0(atom_list):
    # 先遍历一边 然后得出xyz的范围
    # 原始坐标列表
    coordinates = atom_list
    # 初始化最大最小值
    max_x = max_y = max_z = float('-inf')
    min_x = min_y = min_z = float('inf')
    # 平移坐标并寻找最大最小值
    for atom, new_x, new_y, new_z in coordinates:
        # 更新最大值
        if new_x > max_x:
            max_x = new_x
        if new_y > max_y:
            max_y = new_y
        if new_z > max_z:
            max_z = new_z
        # 更新最小值
        if new_x < min_x:
            min_x = new_x
        if new_y < min_y:
            min_y = new_y
        if new_z < min_z:
            min_z = new_z
    # 找到最值了 然后平移
    move_x,move_y,move_z = -min_x,-min_y,-min_z
    # 原来数据
    print("原来晶体的xyz范围：")
    print(f"x 值: {min_x} -- {max_x}")
    print(f"y 值: {min_y} -- {max_y}")
    print(f"z 值: {min_z} -- {max_z}")
# 处理对象

move_to_0(cube_list)

原来晶体的xyz范围：
x 值: 0.1541942654666677 -- 26.69045945416667
y 值: 0.7390986988666697 -- 27.693404760266667
z 值: 0.7840848032666656 -- 33.14589244206667


# 输出

In [130]:
ele_dic1 = {'O':'O','Si':'Si','Al':'Al','Li':'Li'}
# 将坐标写回到xyz文件中
def write_xyz_file(filename, coordinates):
    with open(filename, 'w') as f:
        f.write(f"{len(coordinates)}\n")
        f.write("Atoms\n")
        for atom, x, y, z in coordinates:
            f.write(f"{ele_dic1[atom]}    {x:.6f}    {y:.6f}    {z:.6f}\n")

##########
write_list = cube_list
save_path =  r'C:\Users\81004\Desktop\laso4_VMD.xyz'

path = save_path
############
# 使用示例
write_xyz_file(path, write_list)
print(f"Filtered coordinates saved to {save_path}")
now_time()

Filtered coordinates saved to C:\Users\81004\Desktop\laso4_VMD.xyz
当前时间是: 2025-01-10 03:26:35.364311


In [132]:
# 调整元素的顺序 以满足dat文件的要求
# 将坐标写回到dat文件中
ele_dic = {'O':1,'Si':2,'Al':3,'Li':4}

save_path_dat = r'C:\Users\81004\Desktop\laso4_MD.dat'

# 按照元素种类排序
sorted_coordinates = sorted(write_list, key=lambda x: ele_dic.get(x[0], float('inf')))

In [134]:
len(ele_dic)

4

In [136]:
8.535*10

85.35

In [138]:
5.199*16

83.184

In [140]:
8.96*9

80.64000000000001

In [142]:

############
def write_dat_file(filename, coordinates):
    with open(filename, 'w') as f:
        f.write('# Translate data using python code by Yajiao Zhang.\n')
        f.write(f"{len(cube_list)} atoms \n")
        f.write(f"{len(ele_dic)} atom types\n")
        f.write(f"    0.000000    {len_x:.6f}  xlo xhi\n")
        f.write(f"    0.000000    {len_y:.6f}  ylo yhi\n")
        f.write(f"    0.000000    {len_z:.6f}  zlo zhi\n")
        f.write('    0.000000     0.000000     0.000000 xy xz yz\n')
        f.write('\n')
        f.write("Atoms\n\n")
        count_num = 1
        for atom, x, y, z in coordinates:
            
            f.write(f"    {count_num}  {ele_dic[atom]}  0.00   {x:.6f}    {y:.6f}    {z:.6f}\n")
            count_num+=1
##########
write_list = cube_list

############
# 使用示例
write_dat_file(save_path_dat, sorted_coordinates)
print(f"Filtered coordinates saved to {save_path_dat}")
now_time()

Filtered coordinates saved to C:\Users\81004\Desktop\laso4_MD.dat
当前时间是: 2025-01-10 03:26:45.365442


In [98]:
sorted_coordinates[:10]

[['O', 9.630556601333334, 3.832615548833333, 3.9950267610333334],
 ['O', 5.394330212933333, 8.652143524433335, 3.9950267610333334],
 ['O', 5.394330212933333, 1.5199065126333338, 5.069461589933333],
 ['O', 9.630556601333334, 6.339434488333334, 5.069461589933333],
 ['O', 12.469293676833333, 2.452045180933334, 7.536831349333333],
 ['O', 8.233067288433332, 7.271573156633334, 7.536831349333333],
 ['O', 8.233067288433332, 4.7647542171333335, 6.462396520433334],
 ['O', 12.469293676833333, 9.584282192833333, 6.462396520433334],
 ['O', 11.183998487333334, 1.6504269136333338, 4.767067450633334],
 ['O', 6.947772098833333, 6.469954889333334, 4.767067450633334]]